In [13]:
import pandas as pd

# Dosyaları yükle ve sütun isimlerini göster
m2m100 = pd.read_csv('m2m100_translations_500.csv')
nllb = pd.read_csv('nllb_results_nllb-200-distilled-600M.csv')
helsinki = pd.read_csv('translated_sentences_final.csv')

print("m2m100 sütunları:", m2m100.columns.tolist())
print("nllb sütunları:", nllb.columns.tolist())
print("helsinki sütunları:", helsinki.columns.tolist())

m2m100 sütunları: ['Orijinal', 'Çeviri', 'Süre (s)']
nllb sütunları: ['orijinal', 'çeviri', 'süre']
helsinki sütunları: ['Sentence', 'Turkish_Translation']


In [27]:
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
import nltk
from nltk.tokenize import word_tokenize
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output
from bert_score import score
import warnings
warnings.filterwarnings('ignore')

# Gerekli paketlerin yüklenmesi
!pip install bert-score rouge-score nltk pandas numpy ipywidgets
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Verilerin yüklenmesi (ÖRNEK - KENDİ DOSYALARINIZA GÖRE DÜZENLEYİN)
# Bu kısımda kendi dosya yapınıza uygun düzenlemeler yapmalısınız
try:
    # Örnek yükleme - gerçek dosya yapınıza göre değiştirin
    m2m100 = pd.read_csv('m2m100_translations_500.csv')
    nllb = pd.read_csv('nllb_results_nllb-200-distilled-600M.csv')
    helsinki = pd.read_csv('translated_sentences_final.csv')
    reference_df = pd.read_csv('reference_translations.csv')  # Türkçe referans çeviriler

    # Sütun isimlerini düzenleme (kendi dosyalarınıza göre ayarlayın)
    data = pd.DataFrame({
        'reference': reference_df['turkce_ceviri'],  # Türkçe referans çeviriler
        'm2m100': m2m100['Çeviri'],
        'nllb': nllb['çeviri'],
        'helsinki': helsinki['Turkish_Translation']
    }).head(50)  # İlk 50 cümle

except Exception as e:
    print(f"Dosya yükleme hatası: {str(e)}")
    print("Lütfen dosya yollarını ve sütun isimlerini kontrol edin")
    raise

# Türkçe için özel metin temizleme fonksiyonu
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    # Türkçe karakter dönüşümü
    turkish_chars = {'ı':'i', 'ğ':'g', 'ü':'u', 'ş':'s', 'ö':'o', 'ç':'c'}
    for char, replacement in turkish_chars.items():
        text = text.replace(char, replacement)
    # Noktalama temizliği
    for punct in '.,!?;:"()[]{}':
        text = text.replace(punct, ' ')
    return ' '.join(text.split())

# Gelişmiş metrik hesaplama fonksiyonu
def calculate_metrics(reference, hypothesis):
    # Temizleme ve tokenizasyon
    ref_clean = clean_text(reference)
    hyp_clean = clean_text(hypothesis)

    ref_tokens = word_tokenize(ref_clean) if ref_clean else []
    hyp_tokens = word_tokenize(hyp_clean) if hyp_clean else []

    if not ref_tokens or not hyp_tokens:
        return {metric: 0 for metric in ['BLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'METEOR', 'BERTScore']}

    # BLEU
    try:
        smoothie = SmoothingFunction().method4
        bleu = sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothie)
    except:
        bleu = 0

    # ROUGE
    try:
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        rouge_scores = scorer.score(ref_clean, hyp_clean)
    except:
        rouge_scores = {'rouge1': type('', (), {'fmeasure': 0}),
                       'rouge2': type('', (), {'fmeasure': 0}),
                       'rougeL': type('', (), {'fmeasure': 0})}

    # METEOR
    try:
        meteor = meteor_score([ref_tokens], hyp_tokens)
    except:
        meteor = 0

    # BERTScore
    try:
        _, _, bert_f1 = score([hyp_clean], [ref_clean], lang='tr')
        bert_score = bert_f1.mean().item()
    except:
        bert_score = 0

    return {
        'BLEU': bleu,
        'ROUGE-1': rouge_scores['rouge1'].fmeasure,
        'ROUGE-2': rouge_scores['rouge2'].fmeasure,
        'ROUGE-L': rouge_scores['rougeL'].fmeasure,
        'METEOR': meteor,
        'BERTScore': bert_score
    }

# METRİKLERİ HESAPLA
print("Metrikler hesaplanıyor...")
metrics = {model: {metric: [] for metric in ['BLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'METEOR', 'BERTScore']}
           for model in ['m2m100', 'nllb', 'helsinki']}

for i, row in data.iterrows():
    for model in ['m2m100', 'nllb', 'helsinki']:
        if pd.isna(row[model]):
            # Eksik çeviri durumu
            for metric in metrics[model].keys():
                metrics[model][metric].append(0)
        else:
            model_metrics = calculate_metrics(row['reference'], row[model])
            for metric, value in model_metrics.items():
                metrics[model][metric].append(value)

# Ortalama metrikler
avg_metrics = {model: {metric: np.mean(values) for metric, values in metrics[model].items()}
               for model in metrics}
results_df = pd.DataFrame(avg_metrics).T

# İNSAN DEĞERLENDİRMESİ ARAYÜZÜ
human_scores = {model: [] for model in ['m2m100', 'nllb', 'helsinki']}
current_index = 0

# Widgetlar
sentence_display = widgets.Output()
score_buttons = widgets.RadioButtons(
    options=[1, 2, 3, 4, 5],
    description='Puan:',
    disabled=False,
    layout={'width': 'max-content'}
)
model_selector = widgets.Dropdown(
    options=['m2m100', 'nllb', 'helsinki'],
    description='Model:',
    disabled=False
)
next_button = widgets.Button(description="Sonraki", button_style='success')
save_button = widgets.Button(description="Sonuçları Kaydet", button_style='info')
progress = widgets.IntProgress(value=0, max=50, description='İlerleme:')

def display_sentence(index, model):
    with sentence_display:
        clear_output()
        print(f"### Cümle {index+1}/50 ###")
        print("\n🔹 Kaynak Metin:")
        print(data.iloc[index]['reference'])
        print(f"\n🔹 {model.upper()} Çevirisi:")
        print(data.iloc[index][model])
        print("\nDeğerlendirme Kriterleri:")
        print("1: Çok kötü - 3: Orta - 5: Mükemmel")

def on_next_click(b):
    global current_index
    if current_index < 50:
        human_scores[model_selector.value].append(score_buttons.value)
        progress.value += 1
    if current_index < 49:
        current_index += 1
        display_sentence(current_index, model_selector.value)
    else:
        with sentence_display:
            clear_output()
            print("✅ Tüm değerlendirmeler tamamlandı!")
        next_button.disabled = True

def on_save_click(b):
    # İnsan puanlarını ekle
    human_avg = {model: np.mean(scores) if scores else 0 for model, scores in human_scores.items()}
    final_results = results_df.join(pd.DataFrame.from_dict(human_avg, orient='index', columns=['Human_Score']))

    # Sonuçları göster ve kaydet
    print("\n🔍 FİNAL SONUÇLAR:")
    display(final_results)
    final_results.to_csv('translation_model_comparison.csv')
    files.download('translation_model_comparison.csv')
    print("\n✅ Sonuçlar 'translation_model_comparison.csv' olarak kaydedildi ve indirildi!")

# Etkileşimleri ayarla
display_sentence(current_index, model_selector.value)
model_selector.observe(lambda c: display_sentence(current_index, c['new']), names='value')
next_button.on_click(on_next_click)
save_button.on_click(on_save_click)

# Arayüzü göster
display(widgets.VBox([
    widgets.HTML("<h2>🚀 Makine Çevirisi Model Karşılaştırması</h2>"),
    widgets.HBox([model_selector, score_buttons]),
    sentence_display,
    progress,
    widgets.HBox([next_button, save_button])
]))

# Otomatik metrik sonuçlarını göster
print("\n⭐ OTOMATİK METRİK SONUÇLARI:")
display(results_df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Metrikler hesaplanıyor...



⭐ OTOMATİK METRİK SONUÇLARI:


,BLEU,ROUGE-1,ROUGE-2,ROUGE-L,METEOR,BERTScore
m2m100,0.430166,0.721952,0.520612,0.717190,0.674235,0.885033
nllb,0.447918,0.736417,0.555069,0.730703,0.702592,0.886586
helsinki,1.000000,1.000000,1.000000,1.000000,0.998598,1.000000



🔍 FİNAL SONUÇLAR:


...


✅ Sonuçlar 'translation_model_comparison.csv' olarak kaydedildi ve indirildi!

🔍 FİNAL SONUÇLAR:


,BLEU,ROUGE-1,ROUGE-2,ROUGE-L,METEOR,BERTScore,Human_Score
m2m100,0.430166,0.721952,0.520612,0.717190,0.674235,0.885033,4.703704
nllb,0.447918,0.736417,0.555069,0.730703,0.702592,0.886586,5.000000
helsinki,1.000000,1.000000,1.000000,1.000000,0.998598,1.000000,5.000000


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Sonuçlar 'translation_model_comparison.csv' olarak kaydedildi ve indirildi!
